# 제3 고지 : 고차 미분 계산
## STEP 30 : 고차 미분(준비 편)

현재의 DeZero는 미분을 자동으로 계산할 수 있지만, **1차 미분 한정**이다. 그래서 이번 단계에서는 **2차미분, 나아가 3차,4차미분 등 모든 고차미분까지 자동으로 계산**할 수 있도록 확장한다.  

우선 현재 구현 부터 천천히 살펴보도록 하자.


### 30.1 확인 1 : Variable 인스턴스 변수
```python
class Variable:
    __array_priority__ = 200

    def __init__(self, data, name=None):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError("{} is not supported".format(type(data)))

        self.data = data
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0
    ...
```
현재 `Variable` 클래스에는 여러 인스턴스 변수가 존재한다. 이 중 `data` 와 `grad` 는 순전파와 역전파에 사용되며 둘 모두 `ndarray` 인스턴스를 저장한다.

![image](../assets/%EA%B7%B8%EB%A6%BC%2030-1.png)

이를 부각하기 위해 위의 그림과 같이 표현하며 `data` 와 `grad`가 `ndarray`를 참조하는 경우 다음과 같이 그린다. 

![image](../assets/%EA%B7%B8%EB%A6%BC%2030-2.png)

위의 왼쪽 그림은 `x=Variable(np.array(2.0))` 과 같은 코드 실행시를 나타내며, 추가적으로 `x.backward(); x.grad=np.array(1.0)` 까지 실행하면 오른쪽 그림의 상태가 된다.

### 30.2 확인 2 : Function 클래스

```python
class Function:
    def __call__(self, *inputs):
        inputs = [as_variable(x) for x in inputs]
        #####################################
        # 1. 순전파 계산(메인 처리)
        xs = [x.data for x in i처puts]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]
        #####################################

        if Config.enable_backprop:
            self.generation = max([x.generation for x in inputs])
            #####################################
            # 2. '연결'을 만듦
            for output in outputs:
                output.set_creator(self)
            self.inputs = inputs
            self.outputs = [weakref.ref(output) for output in outputs]
            #####################################

        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()
```

`Function` 클래스의 `__call__()`를 다시 한번 살펴보면, 두가지 단계로 이뤄져 있음을 확인할 수 있다.

1. `Variable` 인스턴스 변수인 data를 꺼내 리스트 `xs` 로 모은후 `forward(*xs)` 를 호출하여 구쳬적인 계산을 수행한다.
2. `set_creator()`를 통해  `Variable` 과 `Function`의 '관계'가 만들어진다. 

주목할 것은 DeZero의 모든 함수는 `Function` 클래스를 상속하고, 구체적인 계산은 `forward()`에서 구현한다. 예를 들어, `Sin` 함수는 다음과 같다. 

```python
class Sin(Function):
    def forward(self,x):
        y= np.sin(x)
        return y 
    
    def backward(self,gy):
        x = self.inputs[0].data 
        gx = gy * np.cos(x)
        return gx 

def sin(x):
    return Sin()(x)

x = Variable(np.array(1.0))
y = sin(x)
```

`Sin` 함수를 순전파 계산을 수행하면, `Function` 과 `Variable`의 '동작'은 다음과 같이 시각화 할 수 있다. 

![image](../assets/%EA%B7%B8%EB%A6%BC%2030-3.png)

### 30.3 확인 3 : Variable 클래스의 역전파

마지막으로 역전파 로직을 살펴보면, 

```python
class Variable:
    ...

    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            ###############################################
            # 역전파 계산 (메인 처리)
            gys = [output().grad for output in f.outputs] # 1.
            gxs = f.backward(*gys) # 2. 
            ###############################################
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            ###############################################
            # 역전파 계산 (메인 처리)
            for x, gx in zip(f.inputs, gxs): # 3.
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
            ###############################################

                if x.creator is not None:
                    add_func(x.creator)

            if not retain_grad:
                for y in f.outputs:
                    y().grad = None  # y is weakref

x = Variable(np.array(1.0))
y = sin(x)
y.backward(retrain_grad=True)
```

위의 코드를 `Function` 과 `Variable`의 '동작' 을 시각화 하면 다음과 같다.

![image](../assets/%EA%B7%B8%EB%A6%BC%2030-4.png)
